In [1]:
import itertools
from collections import namedtuple
import math

import pandas as pd
import numpy as np

# Replication of the toy example of the HIPF paper

This notebook successfully replicates the Hierarchical Iterative Proportional Fitting (HIPF) algorithm on a toy example. Both the algorithm and the toy example are taken from the original paper on HIPF: 

> Müller and Axhausen 2011: "Hierarchical IPF: Generating a synthetic population for Switzerland"

## Fixture and expected output

In the following, the input data and expected output data is prepared.

In [2]:
HouseholdType = namedtuple('HouseholdType', ['household_ids', 'a', 'alpha', 'expansion_factors'])
household_types = []
household_types.append(HouseholdType(
    household_ids=range(1, 23), 
    a=True,
    alpha=[True, False, False],
    expansion_factors=[1.37, 1.36, 1.33, 1.28]
))
household_types.append(HouseholdType(
    household_ids=range(23, 44), 
    a=True,
    alpha=[True, False],
    expansion_factors=[1.37, 1.57, 1.61, 1.61]
))
household_types.append(HouseholdType(
    household_ids=range(44, 65), 
    a=True,
    alpha=[False, False, False],
    expansion_factors=[1.37, 0.94, 0.92, 0.75]
))
household_types.append(HouseholdType(
    household_ids=range(65, 81), 
    a=False,
    alpha=[False, False],
    expansion_factors=[0.64, 0.44, 0.45, 0.38]
))
household_types.append(HouseholdType(
    household_ids=range(81, 97), 
    a=False,
    alpha=[True, False, False],
    expansion_factors=[0.64, 0.64, 0.62, 0.66]
))
household_types.append(HouseholdType(
    household_ids=range(97, 109), 
    a=False,
    alpha=[False],
    expansion_factors=[0.64, 0.44, 0.48, 0.38]
))
household_types.append(HouseholdType(
    household_ids=range(109, 120), 
    a=True,
    alpha=[False, False],
    expansion_factors=[1.37, 0.94, 0.97, 0.75]
))
household_types.append(HouseholdType(
    household_ids=range(120, 129), 
    a=True,
    alpha=[False],
    expansion_factors=[1.37, 0.94, 1.01, 0.75]
))
household_types.append(HouseholdType(
    household_ids=range(129, 137), 
    a=False,
    alpha=[True, True, False],
    expansion_factors=[0.64, 0.83, 0.82, 1.00]
))
household_types.append(HouseholdType(
    household_ids=range(137, 145), 
    a=True,
    alpha=[True, True, False],
    expansion_factors=[1.37, 1.77, 1.73, 1.95]
))
household_types.append(HouseholdType(
    household_ids=range(145, 152), 
    a=False,
    alpha=[True, False],
    expansion_factors=[0.64, 0.74, 0.75, 0.82]
))
household_types.append(HouseholdType(
    household_ids=range(152, 159), 
    a=False,
    alpha=[False, False, False],
    expansion_factors=[0.64, 0.44, 0.43, 0.38]
))
household_types.append(HouseholdType(
    household_ids=range(159, 165), 
    a=True,
    alpha=[True],
    expansion_factors=[1.37, 2.19, 2.35, 2.76]
))
household_types.append(HouseholdType(
    household_ids=range(165, 171), 
    a=True,
    alpha=[True, True],
    expansion_factors=[1.37, 2.19, 2.25, 2.75]
))
household_types.append(HouseholdType(
    household_ids=range(171, 174), 
    a=False,
    alpha=[True],
    expansion_factors=[0.64, 1.03, 1.11, 1.41]
))
household_types.append(HouseholdType(
    household_ids=range(174, 176), 
    a=True,
    alpha=[True, True, True],
    expansion_factors=[1.37, 2.19, 2.14, 2.74]
))
household_types.append(HouseholdType(
    household_ids=range(176, 177), 
    a=False,
    alpha=[True, True],
    expansion_factors=[0.64, 1.03, 1.06, 1.40]
))

In [3]:
id_tuples = itertools.chain(*(itertools.product(ht.household_ids, range(1, len(ht.alpha) + 1)) 
                              for ht in household_types))
index = pd.MultiIndex.from_tuples(list(id_tuples), names=['household_id', 'person_id'])
ref_sample = pd.DataFrame(index=index, columns=['a', 'alpha'])
for ht in household_types:
    ref_sample.ix[ht.household_ids[0]: ht.household_ids[-1], 'a'] = ht.a
    for p, alpha in enumerate(ht.alpha):
        ref_sample.loc[(slice(ht.household_ids[0], ht.household_ids[-1]), p + 1), 'alpha'] = alpha

In [4]:
assert ref_sample.groupby(ref_sample.index.get_level_values(0)).a.count().count() == 176

In [5]:
ref_sample.head()

a  alpha
household_id person_id             
1            1          True   True
             2          True  False
             3          True  False
2            1          True   True
             2          True  False

In [6]:
expected_expansion_factors = pd.DataFrame(
    index=ref_sample.groupby(ref_sample.index.get_level_values(0)).count().index.get_level_values(0), 
    columns=[0, 1, 4, 5, 10], 
    dtype=np.float64
)
expected_expansion_factors[0] = 1.
for ht in household_types:
    for household_id in ht.household_ids:
        expected_expansion_factors.ix[household_id, 1] = ht.expansion_factors[0]
        expected_expansion_factors.ix[household_id, 4] = ht.expansion_factors[1]
        expected_expansion_factors.ix[household_id, 5] = ht.expansion_factors[2]
        expected_expansion_factors.ix[household_id, 10] = ht.expansion_factors[3]
expected_expansion_factors.head()

,0,1,4,5,10
household_id,,,,,
1,1.0,1.37,1.36,1.33,1.28
2,1.0,1.37,1.36,1.33,1.28
3,1.0,1.37,1.36,1.33,1.28
4,1.0,1.37,1.36,1.33,1.28
5,1.0,1.37,1.36,1.33,1.28


### Test helper

In [7]:
from pandas.util.testing import assert_series_equal

def assert_expansion_factors_equal(expected_expansion_factors, expansion_factors):
    assert_series_equal(expected_expansion_factors, expansion_factors, check_less_precise=1)

## Running HIPF

In [8]:
expansion_factors = pd.DataFrame(
    index=ref_sample.groupby(ref_sample.index.get_level_values(0)).count().index.get_level_values(0), 
    data=1.0, 
    columns=[0], 
    dtype=np.float64
)

In [9]:
expansion_factors.head()

,0
household_id,
1,1.0
2,1.0
3,1.0
4,1.0
5,1.0


In [10]:
assert_expansion_factors_equal(expected_expansion_factors[0], expansion_factors[0])

### First Iteration

In [11]:
def fit_households(expansion_factors, total_car):
    new_expansion_factors = expansion_factors.copy()
    for household_id in expansion_factors.index:
        ah = ref_sample.loc[(household_id, 1), 'a']
        mask = ref_sample.groupby(ref_sample.index.get_level_values(0)).a.first() == ah
        new_expansion_factors[household_id] = total_car[ah] / expansion_factors[mask].sum()
    return new_expansion_factors

In [12]:
total_car = pd.Series(index=[True, False], data=[145, 45])
expansion_factors[1] = fit_households(expansion_factors[0], total_car)

In [13]:
assert_expansion_factors_equal(expected_expansion_factors[1], expansion_factors[1])

### Second Iteration

In [14]:
def expand_expansion_factors_to_person(expansion_factors):
    person_expansion_factors = expansion_factors.copy()
    person_expansion_factors = pd.DataFrame(person_expansion_factors)
    person_expansion_factors['person_id'] = 1
    person_expansion_factors.set_index('person_id', append=True, inplace=True)
    person_expansion_factors = person_expansion_factors.reindex(ref_sample.index)
    person_expansion_factors.fillna(method='ffill', inplace=True)
    return person_expansion_factors[expansion_factors.name]

In [15]:
expansion_factors_p = expand_expansion_factors_to_person(expansion_factors[1])

In [16]:
expansion_factors_p.head()

household_id  person_id
1             1            1.367925
              2            1.367925
              3            1.367925
2             1            1.367925
              2            1.367925
Name: 1, dtype: float64

In [17]:
assert math.isclose(expansion_factors_p.sum() - 8.27, 434, abs_tol=0.01)
assert math.isclose(expansion_factors_p[ref_sample.alpha == True].sum() + 85.18, 227, abs_tol=0.01)

### Third Iteration

In [18]:
def fit_person(expansion_factors, total_employment):
    new_expansion_factors = expansion_factors.copy()
    for person_id in expansion_factors.index:
        alphah = ref_sample.loc[person_id, 'alpha']
        mask = ref_sample.alpha == alphah
        new_expansion_factors.loc[person_id] = expansion_factors_p.loc[person_id] * total_employment[alphah] / expansion_factors[mask].sum()
    return new_expansion_factors

In [19]:
total_employment = {True: 227, False: 207}
new_expansion_factors_p = fit_person(expansion_factors_p, total_employment)

In [20]:
assert math.isclose(new_expansion_factors_p.sum(), 434, abs_tol=0.01)
assert math.isclose(new_expansion_factors_p[ref_sample.alpha == True].sum(), 227, abs_tol=0.01)

### Forth Iteration

In [21]:
def aggregate_person_expansion_factors_to_household(person_expansion_factors):
    return person_expansion_factors.groupby(person_expansion_factors.index.get_level_values(0)).mean()

In [22]:
expansion_factors[4] = aggregate_person_expansion_factors_to_household(new_expansion_factors_p)
assert_expansion_factors_equal(expected_expansion_factors[4], expansion_factors[4])

### Fifth Iteration

In [23]:
from itertools import filterfalse

from numpy.polynomial import Polynomial


def rescale_expansion_factors(expansion_factors):
    largest_household_size = ref_sample.groupby(ref_sample.index.get_level_values(0)).alpha.count().max()
    Fp = [expansion_factors[ref_sample.groupby(ref_sample.index.get_level_values(0)).alpha.count() == p].sum()
          for p in range(0, largest_household_size + 1)]
    polynom = [(190 / 434 * p - 1) * Fp[p] for p in range(0, largest_household_size + 1)]
    roots = Polynomial(polynom).roots()
    dx = list(filter(lambda x: np.real(x) > 0, filterfalse(lambda x: np.iscomplex(x), roots)))
    assert len(dx) == 1
    d = np.real(dx[0])
    c = 190 / sum([Fp[p] * d ** p for p in range(1, largest_household_size + 1)])
    fhprime_by_fh = {p: c * d ** p for p in range(1, largest_household_size + 1)}
    
    new_expansion_factors = expansion_factors.copy()
    for household_id in expansion_factors.index:
        household_size = ref_sample.groupby(ref_sample.index.get_level_values(0)).alpha.count()[household_id]
        new_expansion_factors[household_id] = fhprime_by_fh[household_size] * expansion_factors[household_id]
    return new_expansion_factors
    

In [24]:
expansion_factors[5] = rescale_expansion_factors(expansion_factors[4])
assert_expansion_factors_equal(expected_expansion_factors[5], expansion_factors[5])

### Iteration 6-10

In [25]:
expansion_factors[6] = fit_households(expansion_factors[5], total_car)
expansion_factors_p = expand_expansion_factors_to_person(expansion_factors[6])
new_expansion_factors_p = fit_person(expansion_factors_p, total_employment)
expansion_factors[9] = aggregate_person_expansion_factors_to_household(new_expansion_factors_p)
expansion_factors[10] = rescale_expansion_factors(expansion_factors[9])

In [26]:
assert_expansion_factors_equal(expected_expansion_factors[10], expected_expansion_factors[10])